# Praktikum 1
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense

## Preprocessing

## Model

### RNN

In [ ]:
rnn = Sequential()

### LSTM

In [ ]:
lstm = Sequential()

### CNN

In [ ]:
cnn = Sequential()

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
f1_score_test = f1_score(y_true, y_pred, average='macro')
accuracy_score_test = accuracy_score(y_true, y_pred)
confusion_matrix_test = confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
np.set_printoptions(precision=2)
print('Confusion matrix')
print(cm)
plt.figure()
plot_confusion_matrix(confusion_matrix_test)